# Dog-Friendly Neighbourhoods of Stockholm

<img src="https://miro.medium.com/max/1800/1*Ajbb76yGEqKPRdbp-rrn4g.jpeg" alt="dogswelcome" align="left" width=600>

<p>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Hello, My name is Liuba and I live in Gothenburg with my dog and sidekick called Watson. We live in a neighbourhood called Olskroken. When I was buying my appartment I chose this neighbourhood for it's dog-friendliness (before I even met Watson!).</p>
<p><strong>What does a dog-friendly neighbourhood mean?</strong> From my point of view a neighbourhood can be called dog-friendly if it has the following attributes:</p>
<ul>
<li>A forest or a park</li>
<li>An Animal Hospital</li>
<li>A Dog Park (Called "hundrasgarden" in Swedish)</li>
<li>Doggy Daycare (Called "hunddagis" in Swedish)</li>
<li>A Pet Store</li>
<li>A Pet Salon</li>
<li>Dog-friendly cafes and restaurants</li>
</ul>
<p>It is not necessary for a neighbourhood to have all the attributes mentioned above to be called dog-friendly but the more checkboxes it ticks the higher it would be on my list.</p>
<p><strong>Now to the problem and goal of this project</strong>: I am looking into moving to Stockholm and I would like to find a dog-friendly area to live in. The goal of this project would be to determine and compare dog-friendly neighbourhoods in Stockholm.</p>
<p>I believe the end results of this analysis would be beneficial to any dog owner living in Stockholm or someone who wants to move with their furry buddy to this city.</p>
<p>I will be using Foursquare API to retrieve venue information and any data that I can scrape on the&nbsp;Wikipedia or the internet about the neighbourhoods of Stockholm.</p>

In [1]:
# Downloading the dependencies that will be required
import pandas as pd
import numpy as np  # library to handle data in a vectorized manner
import json # library to handle JSON files
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import matplotlib.cm as cm # Matplotlib and associated plotting modules
import matplotlib.colors as colors
from sklearn.cluster import KMeans # import k-means from clustering stage
import folium # map rendering library

print('Libraries imported.')

In [106]:
data = pd.read_csv("C:/Users/lcuzacov/Desktop/GitHub/Coursera_Capstone/stockholm_districts.csv")


In [107]:
data["lat"] = float(1)
data["long"] = float(1)

In [134]:
for i in range(115):
    address = data.iloc[i+j,2] + ", "+ data.iloc[i+j,0]
    geolocator = Nominatim(user_agent="explorer")
    location = geolocator.geocode(address)
    data.set_value(i+j,'lat', float(location.latitude))
    data.set_value(i+j,'long', float(location.longitude))
    #print(i,float(location.latitude)," ",float(location.longitude))

In [136]:
address = 'Stockholm,Sweden'

geolocator = Nominatim(user_agent="tor_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Stockholm are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Stockholm are 59.3251172, 18.0710935.


In [141]:
# Creating a map of Toronto using latitude and longitude values
map_stkhlm = folium.Map(location=[latitude, longitude], zoom_start=12)

# Adding markers to map
for lat, lng, borough, district in zip(data['lat'], data['long'], data['Borough'], data['District']):
    label = '{}, {}'.format(district, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=10,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_stkhlm)  
    
map_stkhlm